In [ ]:
%%sql
CREATE TABLE delta.`abfss://silverlayerdata@energyscotish.dfs.core.windows.net/metadata_table` (
    TableName STRING,
    LastProcessedTimestamp TIMESTAMP
) USING delta;


StatementMeta(, , -1, SessionError, , SessionError)

AVAILABLE_WORKSPACE_CAPACITY_EXCEEDED: Livy session has failed. Session state: Error. Error code: AVAILABLE_WORKSPACE_CAPACITY_EXCEEDED. Your job requested 12 vcores. However, the workspace only has 0 vcores available out of quota of 12 vcores for node size family [MemoryOptimized]. Try ending the running job(s), reducing the numbers of vcores requested or increasing your vcore quota. https://learn.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-concepts#workspace-level Source: User.

In [42]:
#%%sql
#INSERT INTO delta.`abfss://silverlayerdata@energyscotish.dfs.core.windows.net/metadata_table`
#VALUES
   #('billing_silver', '1970-01-01 00:00:00'),
    #('customers_silver', '1970-01-01 00:00:00'),
    #('regions_silver', '1970-01-01 00:00:00'),
    #('iot_silver', '1970-01-01 00:00:00');


StatementMeta(TestCluster, 0, 42, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
%%sql
MERGE INTO delta.`abfss://silverlayerdata@energyscotish.dfs.core.windows.net/metadata_table` AS target
USING (
    SELECT 'billing_silver' AS TableName, MAX(b.last_updated) AS LastProcessedTimestamp
    FROM delta.`abfss://silverlayerdata@energyscotish.dfs.core.windows.net/billing_data` AS b
    UNION ALL
    SELECT 'customers_silver', MAX(c.last_updated)
    FROM delta.`abfss://silverlayerdata@energyscotish.dfs.core.windows.net/customers_data` AS c
    UNION ALL
    SELECT 'regions_silver', MAX(r.last_updated)
    FROM delta.`abfss://silverlayerdata@energyscotish.dfs.core.windows.net/regions_data` AS r
    UNION ALL
    SELECT 'iot_silver', MAX(i.last_updated)
    FROM delta.`abfss://silverlayerdata@energyscotish.dfs.core.windows.net/iot_data` AS i
) AS source
ON target.TableName = source.TableName
WHEN MATCHED THEN
    UPDATE SET
        target.LastProcessedTimestamp = source.LastProcessedTimestamp
WHEN NOT MATCHED THEN
    INSERT (TableName, LastProcessedTimestamp)
    VALUES (source.TableName, source.LastProcessedTimestamp);
